In [1]:
from datetime import datetime
import datetime as dt
import os
from operator import itemgetter
import shutil 
import pickle
import time

from shapely.geometry import Point, Polygon
from osgeo import osr
from pyproj import Proj
from fiona.crs import from_epsg
from sklearn.cluster import KMeans
import geopandas as gpd
from affine import Affine
import xarray as xr
import numpy as np
from rasterio.features import rasterize
import fiona
import glob
from dask.distributed import Client
from datacube.drivers.netcdf import write_dataset_to_netcdf
from datacube.api import GridWorkflow

from madmex.wrappers import gwf_query
from madmex.util.xarray import to_float, to_int

2020-02-01 13:53:27,096 - datacube.drivers.driver_cache - driver_cache safe_load: Failed to resolve driver datacube.plugins.index::s3aio_index
2020-02-01 13:53:27,099 - datacube.drivers.driver_cache - driver_cache safe_load: Error was: UndefinedEnvironmentName("'extra' does not exist in evaluation environment.",)
distributed.nanny - WARNING - Restarting worker
distributed.nanny - WARNING - Restarting worker
distributed.nanny - WARNING - Restarting worker
distributed.nanny - WARNING - Restarting worker
distributed.nanny - WARNING - Restarting worker
distributed.comm.tcp - WARNING - Closing dangling stream in <TCP  local=tcp://127.0.0.1:37006 remote=tcp://127.0.0.1:40395>
distributed.comm.tcp - WARNING - Closing dangling stream in <TCP  local=tcp://127.0.0.1:37008 remote=tcp://127.0.0.1:40395>
distributed.comm.tcp - WARNING - Closing dangling stream in <TCP  local=tcp://127.0.0.1:37012 remote=tcp://127.0.0.1:40395>
distributed.comm.tcp - WARNING - Closing dangling stream in <TCP  local=t

In [2]:
client = Client(n_workers=5,memory_limit='100GB')#use also threads_per_worker=1 
#https://stackoverflow.com/questions/53936237/managing-worker-memory-on-a-dask-localcluster

In [3]:
client

Client Scheduler: tcp://127.0.0.1:40395 Dashboard: http://127.0.0.1:8787/status,Cluster Workers: 5 Cores: 35 Memory: 500.00 GB


In [4]:
client.restart()

Client Scheduler: tcp://127.0.0.1:40395 Dashboard: http://127.0.0.1:8787/status,Cluster Workers: 5 Cores: 35 Memory: 500.00 GB


In [5]:
region = 'shp_puntos_martinez_simplify_0'
name_of_product = 's2_l2a_10m_scl_mexico'
begin = '2018-01-01'
end = '2019-12-31'
gwf_kwargs = {'region': region, 'product': name_of_product, 'begin': begin, 'end':end}
iterable = gwf_query(**gwf_kwargs)

In [6]:
lista_iter = list(iterable)

In [7]:
[lista_iter[x][0] for x in range(0,len(lista_iter))]

[(40, -31),
 (41, -31),
 (40, -30),
 (41, -30),
 (40, -32),
 (39, -31),
 (39, -30),
 (41, -32)]

# 1) Se elige el primer dc_tile de lista_iter para la generación de segmentos clusterizados

In [8]:
date_today = datetime.today().strftime("%m-%d-%Y")

In [9]:
date_today

'02-01-2020'

In [10]:
index=0

In [11]:
var=lista_iter[0]

In [12]:
dc_tile = '%d_%d' % (var[0][0], var[0][1])

In [13]:
var

((40, -31), Tile<sources=<xarray.DataArray (time: 188)>
 array([(Dataset <id=a0da98d8-3b8e-4f9c-b7fe-3bb352072e06 type=s2_l2a_10m_scl_mexico location=file:///shared_volume/datacube/datacube_ingest/s2_l2a_10m_scl_s3/mexico/s2_l2a_10m_scl_s3_40_-31_20181004170824000000.nc>,),
        (Dataset <id=0320c527-1c9d-4bd7-b050-f4dbc6ffd87b type=s2_l2a_10m_scl_mexico location=file:///shared_volume/datacube/datacube_ingest/s2_l2a_10m_scl_s3/mexico/s2_l2a_10m_scl_s3_40_-31_20181006170522000000.nc>,),
        (Dataset <id=042502ef-409b-4600-b234-3155de11f780 type=s2_l2a_10m_scl_mexico location=file:///shared_volume/datacube/datacube_ingest/s2_l2a_10m_scl_s3/mexico/s2_l2a_10m_scl_s3_40_-31_20181009171234000000.nc>,),
        (Dataset <id=07a02783-ebe2-421c-9614-1061c3d5c26f type=s2_l2a_10m_scl_mexico location=file:///shared_volume/datacube/datacube_ingest/s2_l2a_10m_scl_s3/mexico/s2_l2a_10m_scl_s3_40_-31_20181014171122000000.nc>,),
        (Dataset <id=99d4e8a9-f239-4d1e-8024-6ffd32b59a72 type=s2_l2

In [14]:
sr_0 = GridWorkflow.load(var[1],dask_chunks={'x': 1600, 'y': 1600,'time': 60})

In [15]:
sr_0

<xarray.Dataset>
Dimensions:   (time: 188, x: 5002, y: 5002)
Coordinates:
  * time      (time) datetime64[ns] 2018-10-04T17:08:24 ... 2019-09-26T17:08:27.160000
  * y         (y) float64 9.261e+05 9.261e+05 9.261e+05 ... 8.761e+05 8.761e+05
  * x         (x) float64 2.978e+06 2.978e+06 2.978e+06 ... 3.028e+06 3.028e+06
Data variables:
    blue      (time, y, x) uint16 dask.array<chunksize=(60, 1600, 1600), meta=np.ndarray>
    green     (time, y, x) uint16 dask.array<chunksize=(60, 1600, 1600), meta=np.ndarray>
    red       (time, y, x) uint16 dask.array<chunksize=(60, 1600, 1600), meta=np.ndarray>
    nir       (time, y, x) uint16 dask.array<chunksize=(60, 1600, 1600), meta=np.ndarray>
    pixel_qa  (time, y, x) uint16 dask.array<chunksize=(60, 1600, 1600), meta=np.ndarray>
Attributes:
    crs:      PROJCS["unnamed",GEOGCS["WGS 84",DATUM["unknown",SPHEROID["WGS8...

In [16]:
crs = var[1].geobox.crs

In [17]:
sr_0.attrs['geobox'] = var[1].geobox

In [20]:
sr_0 = client.persist(sr_0)

**110gb is enough**

In [21]:
sr_0

<xarray.Dataset>
Dimensions:   (time: 188, x: 5002, y: 5002)
Coordinates:
  * time      (time) datetime64[ns] 2018-10-04T17:08:24 ... 2019-09-26T17:08:27.160000
  * y         (y) float64 9.261e+05 9.261e+05 9.261e+05 ... 8.761e+05 8.761e+05
  * x         (x) float64 2.978e+06 2.978e+06 2.978e+06 ... 3.028e+06 3.028e+06
Data variables:
    blue      (time, y, x) uint16 dask.array<chunksize=(60, 1600, 1600), meta=np.ndarray>
    green     (time, y, x) uint16 dask.array<chunksize=(60, 1600, 1600), meta=np.ndarray>
    red       (time, y, x) uint16 dask.array<chunksize=(60, 1600, 1600), meta=np.ndarray>
    nir       (time, y, x) uint16 dask.array<chunksize=(60, 1600, 1600), meta=np.ndarray>
    pixel_qa  (time, y, x) uint16 dask.array<chunksize=(60, 1600, 1600), meta=np.ndarray>
Attributes:
    crs:      PROJCS["unnamed",GEOGCS["WGS 84",DATUM["unknown",SPHEROID["WGS8...
    geobox:   GeoBox(Geometry({'type': 'Polygon', 'coordinates': [[(-97.40609...

In [22]:
#write_dataset_to_netcdf(sr_0.compute(),'/shared_volume/sr_0.nc')

In [23]:
sr_0 = sr_0.apply(func=to_float, keep_attrs=True)

In [24]:
sr_1 = sr_0.where(sr_0.pixel_qa.isin([2,4,5,6,7,8,11]))

In [25]:
sr_1['ndvi'] = ((sr_1.nir - sr_1.red) / (sr_1.nir + sr_1.red)) * 10000

In [26]:
sr_1['ndvi'].attrs['nodata'] = 0

In [27]:
sr_1

<xarray.Dataset>
Dimensions:   (time: 188, x: 5002, y: 5002)
Coordinates:
  * time      (time) datetime64[ns] 2018-10-04T17:08:24 ... 2019-09-26T17:08:27.160000
  * y         (y) float64 9.261e+05 9.261e+05 9.261e+05 ... 8.761e+05 8.761e+05
  * x         (x) float64 2.978e+06 2.978e+06 2.978e+06 ... 3.028e+06 3.028e+06
Data variables:
    blue      (time, y, x) float64 dask.array<chunksize=(60, 1600, 1600), meta=np.ndarray>
    green     (time, y, x) float64 dask.array<chunksize=(60, 1600, 1600), meta=np.ndarray>
    red       (time, y, x) float64 dask.array<chunksize=(60, 1600, 1600), meta=np.ndarray>
    nir       (time, y, x) float64 dask.array<chunksize=(60, 1600, 1600), meta=np.ndarray>
    pixel_qa  (time, y, x) float64 dask.array<chunksize=(60, 1600, 1600), meta=np.ndarray>
    ndvi      (time, y, x) float64 dask.array<chunksize=(60, 1600, 1600), meta=np.ndarray>
Attributes:
    crs:      PROJCS["unnamed",GEOGCS["WGS 84",DATUM["unknown",SPHEROID["WGS8...
    geobox:   GeoBox(Geometry({'type': 'Polygon', 'coordinates': [[(-97.40609...

In [28]:
sr_1 = client.persist(sr_1)

In [29]:
sr_1

<xarray.Dataset>
Dimensions:   (time: 188, x: 5002, y: 5002)
Coordinates:
  * time      (time) datetime64[ns] 2018-10-04T17:08:24 ... 2019-09-26T17:08:27.160000
  * y         (y) float64 9.261e+05 9.261e+05 9.261e+05 ... 8.761e+05 8.761e+05
  * x         (x) float64 2.978e+06 2.978e+06 2.978e+06 ... 3.028e+06 3.028e+06
Data variables:
    blue      (time, y, x) float64 dask.array<chunksize=(60, 1600, 1600), meta=np.ndarray>
    green     (time, y, x) float64 dask.array<chunksize=(60, 1600, 1600), meta=np.ndarray>
    red       (time, y, x) float64 dask.array<chunksize=(60, 1600, 1600), meta=np.ndarray>
    nir       (time, y, x) float64 dask.array<chunksize=(60, 1600, 1600), meta=np.ndarray>
    pixel_qa  (time, y, x) float64 dask.array<chunksize=(60, 1600, 1600), meta=np.ndarray>
    ndvi      (time, y, x) float64 dask.array<chunksize=(60, 1600, 1600), meta=np.ndarray>
Attributes:
    crs:      PROJCS["unnamed",GEOGCS["WGS 84",DATUM["unknown",SPHEROID["WGS8...
    geobox:   GeoBox(Geometry({'type': 'Polygon', 'coordinates': [[(-97.40609...

# 2) Cálculo de features por ejemplo el máximo, mínimo del NDVI en toda la serie de tiempo y el promedio NDVI

In [31]:
ndvi_max = sr_1.ndvi.max('time', keep_attrs=True, skipna=True)

In [32]:
ndvi_max

<xarray.DataArray 'ndvi' (y: 5002, x: 5002)>
dask.array<nanmax-aggregate, shape=(5002, 5002), dtype=float64, chunksize=(1600, 1600), chunktype=numpy.ndarray>
Coordinates:
  * y        (y) float64 9.261e+05 9.261e+05 9.261e+05 ... 8.761e+05 8.761e+05
  * x        (x) float64 2.978e+06 2.978e+06 2.978e+06 ... 3.028e+06 3.028e+06
Attributes:
    nodata:   0

In [33]:
ndvi_max = client.persist(ndvi_max)

In [34]:
ndvi_max = ndvi_max.rename('ndvi_max')
ndvi_max.attrs['nodata'] = 0

In [35]:
ndvi_max

<xarray.DataArray 'ndvi_max' (y: 5002, x: 5002)>
dask.array<nanmax-aggregate, shape=(5002, 5002), dtype=float64, chunksize=(1600, 1600), chunktype=numpy.ndarray>
Coordinates:
  * y        (y) float64 9.261e+05 9.261e+05 9.261e+05 ... 8.761e+05 8.761e+05
  * x        (x) float64 2.978e+06 2.978e+06 2.978e+06 ... 3.028e+06 3.028e+06
Attributes:
    nodata:   0

In [36]:
ndvi_min = sr_1.ndvi.min('time', keep_attrs=True, skipna=True)

In [37]:
ndvi_min = client.persist(ndvi_min)

In [38]:
ndvi_min

<xarray.DataArray 'ndvi' (y: 5002, x: 5002)>
dask.array<nanmin-aggregate, shape=(5002, 5002), dtype=float64, chunksize=(1600, 1600), chunktype=numpy.ndarray>
Coordinates:
  * y        (y) float64 9.261e+05 9.261e+05 9.261e+05 ... 8.761e+05 8.761e+05
  * x        (x) float64 2.978e+06 2.978e+06 2.978e+06 ... 3.028e+06 3.028e+06
Attributes:
    nodata:   0

In [39]:
ndvi_min = ndvi_min.rename('ndvi_min')
ndvi_min.attrs['nodata'] = 0

In [40]:
ndvi_min

<xarray.DataArray 'ndvi_min' (y: 5002, x: 5002)>
dask.array<nanmin-aggregate, shape=(5002, 5002), dtype=float64, chunksize=(1600, 1600), chunktype=numpy.ndarray>
Coordinates:
  * y        (y) float64 9.261e+05 9.261e+05 9.261e+05 ... 8.761e+05 8.761e+05
  * x        (x) float64 2.978e+06 2.978e+06 2.978e+06 ... 3.028e+06 3.028e+06
Attributes:
    nodata:   0

In [38]:
sr_0.geobox

GeoBox(5002, 5002, Affine(10.0, 0.0, 2977960.0,
       0.0, -10.0, 926120.0), PROJCS["unnamed",GEOGCS["WGS 84",DATUM["unknown",SPHEROID["WGS84",6378137,6556752.3141]],PRIMEM["Greenwich",0],UNIT["degree",0.0174532925199433]],PROJECTION["Lambert_Conformal_Conic_2SP"],PARAMETER["standard_parallel_1",17.5],PARAMETER["standard_parallel_2",29.5],PARAMETER["latitude_of_origin",12],PARAMETER["central_meridian",-102],PARAMETER["false_easting",2500000],PARAMETER["false_northing",0]])

**check next lines to compute median, percentiles in the future**

In [36]:
#import dask.array as da

In [39]:
#ndvi_min_nan = da.nanmin(sr_1.ndvi, axis=0)

In [40]:
#ndvi_min_nan

dask.array<nanmin-aggregate, shape=(5002, 5002), dtype=float64, chunksize=(1600, 1600), chunktype=numpy.ndarray>

In [41]:
#ndvi_min_nan = client.persist(ndvi_min_nan)

In [42]:
#ndvi_min_nan

dask.array<nanmin-aggregate, shape=(5002, 5002), dtype=float64, chunksize=(1600, 1600), chunktype=numpy.ndarray>

In [43]:
#ndvi_median = sr_1.ndvi.median('time', keep_attrs=True, skipna=True)

In [ ]:
#ndvi_median = client.persist(ndvi_median)

In [ ]:
#ndvi_median

In [ ]:
#ndvi_median = ndvi_median.rename('ndvi_median')
#ndvi_median.attrs['nodata'] = 0

In [ ]:
#ndvi_median

In [41]:
ndvi_mean = sr_1.ndvi.mean('time', keep_attrs=True, skipna=True)

In [42]:
ndvi_mean = client.persist(ndvi_mean)

In [43]:
ndvi_mean

<xarray.DataArray 'ndvi' (y: 5002, x: 5002)>
dask.array<mean_agg-aggregate, shape=(5002, 5002), dtype=float64, chunksize=(1600, 1600), chunktype=numpy.ndarray>
Coordinates:
  * y        (y) float64 9.261e+05 9.261e+05 9.261e+05 ... 8.761e+05 8.761e+05
  * x        (x) float64 2.978e+06 2.978e+06 2.978e+06 ... 3.028e+06 3.028e+06
Attributes:
    nodata:   0

In [44]:
ndvi_mean = ndvi_mean.rename('ndvi_mean')
ndvi_mean.attrs['nodata'] = 0

In [45]:
ndvi_mean

<xarray.DataArray 'ndvi_mean' (y: 5002, x: 5002)>
dask.array<mean_agg-aggregate, shape=(5002, 5002), dtype=float64, chunksize=(1600, 1600), chunktype=numpy.ndarray>
Coordinates:
  * y        (y) float64 9.261e+05 9.261e+05 9.261e+05 ... 8.761e+05 8.761e+05
  * x        (x) float64 2.978e+06 2.978e+06 2.978e+06 ... 3.028e+06 3.028e+06
Attributes:
    nodata:   0

In [46]:
combined = xr.merge([ndvi_mean,
                     ndvi_min,
                     ndvi_max])

In [47]:
combined

<xarray.Dataset>
Dimensions:    (x: 5002, y: 5002)
Coordinates:
  * y          (y) float64 9.261e+05 9.261e+05 9.261e+05 ... 8.761e+05 8.761e+05
  * x          (x) float64 2.978e+06 2.978e+06 2.978e+06 ... 3.028e+06 3.028e+06
Data variables:
    ndvi_mean  (y, x) float64 dask.array<chunksize=(1600, 1600), meta=np.ndarray>
    ndvi_min   (y, x) float64 dask.array<chunksize=(1600, 1600), meta=np.ndarray>
    ndvi_max   (y, x) float64 dask.array<chunksize=(1600, 1600), meta=np.ndarray>

In [48]:
combined.data_vars

Data variables:
    ndvi_mean  (y, x) float64 dask.array<chunksize=(1600, 1600), meta=np.ndarray>
    ndvi_min   (y, x) float64 dask.array<chunksize=(1600, 1600), meta=np.ndarray>
    ndvi_max   (y, x) float64 dask.array<chunksize=(1600, 1600), meta=np.ndarray>

In [49]:
features = list(combined.data_vars)
s = '-'
features_string = s.join(features)

In [50]:
features_string

'ndvi_mean-ndvi_min-ndvi_max'

In [51]:
path_result = '/LUSTRE/MADMEX/tasks/2020/1_clusterization_for_agriculture_labeling/features_computed/' + features_string + '/'

In [52]:
path_result_dc_tile = os.path.join(path_result, date_today 
                                   + '/' + dc_tile + '/')

In [53]:
path_result_dc_tile

'/LUSTRE/MADMEX/tasks/2020/1_clusterization_for_agriculture_labeling/features_computed/ndvi_mean-ndvi_min-ndvi_max/02-01-2020/40_-31/'

In [54]:
if not os.path.exists(path_result_dc_tile):
    os.makedirs(path_result_dc_tile)

In [55]:
nc_filename = path_result_dc_tile + date_today + '_combined_' + dc_tile + '.nc'

In [56]:
nc_filename

'/LUSTRE/MADMEX/tasks/2020/1_clusterization_for_agriculture_labeling/features_computed/ndvi_mean-ndvi_min-ndvi_max/02-01-2020/40_-31/02-01-2020_combined_40_-31.nc'

In [57]:
combined.attrs['crs'] = crs

In [58]:
write_dataset_to_netcdf(combined.compute(),nc_filename)

# 3) Leer segmentación

In [59]:
path_seg = '/shared_volume/segmentation_antares3_results/bis_seg_mex_s2_10m_scl_10_18_09_19_sentinel_2_{}_2019.shp'.format(dc_tile)

In [60]:
path_seg

'/shared_volume/segmentation_antares3_results/bis_seg_mex_s2_10m_scl_10_18_09_19_sentinel_2_40_-31_2019.shp'

In [61]:
with fiona.open(path_seg) as src:
    fc = list(src)

In [62]:
fc_subset=((x['properties']['id'], x['geometry']) for x in fc)

In [63]:
fc_sorted = sorted(fc_subset, key=itemgetter(0))

In [64]:
fc_sorted[0]

(0,
 {'type': 'Polygon',
  'coordinates': [[(2978000.0, 876160.0),
    (2978010.0, 876160.0),
    (2978010.0, 876120.0),
    (2978000.0, 876120.0),
    (2977980.0, 876120.0),
    (2977980.0, 876100.0),
    (2977960.0, 876100.0),
    (2977960.0, 876140.0),
    (2977980.0, 876140.0),
    (2977990.0, 876140.0),
    (2977990.0, 876150.0),
    (2978000.0, 876150.0),
    (2978000.0, 876160.0)]]})

In [65]:
len([k[0] for k in fc_sorted])

538023

**First position of fc_sorted is id, second position is geometry**


In [66]:
iterable_zip = zip([k[1] for k in fc_sorted], [k[0] for k in fc_sorted])

In [67]:
sr_0.affine

Affine(10.0, 0.0, 2977960.0,
       0.0, -10.0, 926120.0)

In [68]:
list(sr_0.affine)

[10.0, 0.0, 2977960.0, 0.0, -10.0, 926120.0, 0.0, 0.0, 1.0]

In [69]:
list(sr_0.affine)[0:6]

[10.0, 0.0, 2977960.0, 0.0, -10.0, 926120.0]

In [70]:
aff = Affine(*list(sr_0.affine)[0:6])

In [71]:
dimensions_dataset = list(sr_0.coords)
list_dimensions = [x for x in dimensions_dataset if x != 'time']
lambda_function = lambda l_netcdf,l_test: l_netcdf[0] if l_netcdf[0] in l_test else l_netcdf[1]
xdim = lambda_function(list_dimensions,['x','longitude'])
ydim = lambda_function(list_dimensions,['y','latitude'])

In [72]:
fc_raster = rasterize(iterable_zip, transform=aff,
                      out_shape=(sr_0.sizes[ydim], sr_0.sizes[xdim]),
                      dtype='float64', fill=np.nan)

In [73]:
fc_raster

array([[5.37885e+05, 5.37885e+05, 5.37282e+05, ..., 5.37673e+05,
        5.37673e+05, 5.37673e+05],
       [5.37885e+05, 5.37885e+05, 5.37282e+05, ..., 5.37673e+05,
        5.37673e+05, 5.37673e+05],
       [5.37885e+05, 5.37885e+05, 5.37282e+05, ..., 5.37673e+05,
        5.37673e+05, 5.37673e+05],
       ...,
       [0.00000e+00, 0.00000e+00, 0.00000e+00, ..., 8.11000e+02,
        8.11000e+02, 8.11000e+02],
       [0.00000e+00, 0.00000e+00, 1.00000e+00, ..., 8.11000e+02,
        8.11000e+02, 8.11000e+02],
       [0.00000e+00, 0.00000e+00, 1.00000e+00, ..., 8.11000e+02,
        8.11000e+02, 8.11000e+02]])

In [74]:
fc_raster = fc_raster.astype(int)

In [75]:
fc_raster

array([[537885, 537885, 537282, ..., 537673, 537673, 537673],
       [537885, 537885, 537282, ..., 537673, 537673, 537673],
       [537885, 537885, 537282, ..., 537673, 537673, 537673],
       ...,
       [     0,      0,      0, ...,    811,    811,    811],
       [     0,      0,      1, ...,    811,    811,    811],
       [     0,      0,      1, ...,    811,    811,    811]])

In [76]:
fc_dataarray = xr.DataArray(fc_raster, dims=[ydim, xdim], name='features_id')

In [77]:
fc_dataarray

<xarray.DataArray 'features_id' (y: 5002, x: 5002)>
array([[537885, 537885, 537282, ..., 537673, 537673, 537673],
       [537885, 537885, 537282, ..., 537673, 537673, 537673],
       [537885, 537885, 537282, ..., 537673, 537673, 537673],
       ...,
       [     0,      0,      0, ...,    811,    811,    811],
       [     0,      0,      1, ...,    811,    811,    811],
       [     0,      0,      1, ...,    811,    811,    811]])
Dimensions without coordinates: y, x

# 4) Construcción de dataframes para pasar al kmeans

In [78]:
combined = xr.merge([fc_dataarray,combined])

In [79]:
combined

<xarray.Dataset>
Dimensions:      (x: 5002, y: 5002)
Coordinates:
  * x            (x) float64 2.978e+06 2.978e+06 ... 3.028e+06 3.028e+06
  * y            (y) float64 9.261e+05 9.261e+05 ... 8.761e+05 8.761e+05
Data variables:
    features_id  (y, x) int64 537885 537885 537282 537282 ... 811 811 811 811
    ndvi_mean    (y, x) float64 dask.array<chunksize=(1600, 1600), meta=np.ndarray>
    ndvi_min     (y, x) float64 dask.array<chunksize=(1600, 1600), meta=np.ndarray>
    ndvi_max     (y, x) float64 dask.array<chunksize=(1600, 1600), meta=np.ndarray>

In [80]:
combined = combined.stack(z=(xdim, ydim))
combined = combined.reset_index('z')
combined = combined.drop([xdim,ydim])


In [81]:
combined

<xarray.Dataset>
Dimensions:      (z: 25020004)
Dimensions without coordinates: z
Data variables:
    features_id  (z) int64 537885 537885 537885 537282 ... 811 811 811 811
    ndvi_mean    (z) float64 dask.array<chunksize=(2000800,), meta=np.ndarray>
    ndvi_min     (z) float64 dask.array<chunksize=(2000800,), meta=np.ndarray>
    ndvi_max     (z) float64 dask.array<chunksize=(2000800,), meta=np.ndarray>

In [82]:
combined = combined.where((np.isfinite(combined['features_id'])), drop=True)

In [83]:
combined

<xarray.Dataset>
Dimensions:      (z: 25020004)
Dimensions without coordinates: z
Data variables:
    features_id  (z) float64 5.379e+05 5.379e+05 5.379e+05 ... 811.0 811.0 811.0
    ndvi_mean    (z) float64 dask.array<chunksize=(2000800,), meta=np.ndarray>
    ndvi_min     (z) float64 dask.array<chunksize=(2000800,), meta=np.ndarray>
    ndvi_max     (z) float64 dask.array<chunksize=(2000800,), meta=np.ndarray>

In [84]:
df = combined.to_dataframe()

**See recommendation of using scatter with client, screenshot taken in desktop of lap's Erick**

In [85]:
df.head()

,features_id,ndvi_mean,ndvi_min,ndvi_max
z,,,,
0,537885.0,7497.716885,4984.894260,8746.208291
1,537885.0,7799.393099,5715.453802,8825.995807
2,537885.0,7705.309446,5475.648612,8677.685950
3,537282.0,6898.643743,4761.154856,8443.597219
4,537282.0,6475.216811,4147.121535,8202.472436


In [86]:
len(df.index)

25020004

In [87]:
features

['ndvi_mean', 'ndvi_min', 'ndvi_max']

**A partir de aquí se puede cerrar la conexión con client de dask**

In [30]:
#client.close()

In [88]:
df2 = df.groupby('features_id').agg({features[0]: 'median',
                                    features[1]: 'median',
                                    features[2]: 'median'})

In [89]:
df2.head()

,ndvi_mean,ndvi_min,ndvi_max
features_id,,,
0.0,5713.245835,2357.071214,8822.684853
1.0,5074.610973,2184.786384,8533.178114
2.0,4781.271702,1456.722660,8611.947284
3.0,6398.678016,2108.266427,8448.201008
4.0,4987.369208,1551.788250,7403.809376


In [90]:
df2 = df2.reset_index('features_id')

In [91]:
df2.head()

,features_id,ndvi_mean,ndvi_min,ndvi_max
0,0.0,5713.245835,2357.071214,8822.684853
1,1.0,5074.610973,2184.786384,8533.178114
2,2.0,4781.271702,1456.722660,8611.947284
3,3.0,6398.678016,2108.266427,8448.201008
4,4.0,4987.369208,1551.788250,7403.809376


In [92]:
len(df2.index)

538023

**Convert to geopandas dataframe fc of bis segmentation**

In [93]:
var[1].geobox.crs.crs_str

'PROJCS["unnamed",GEOGCS["WGS 84",DATUM["unknown",SPHEROID["WGS84",6378137,6556752.3141]],PRIMEM["Greenwich",0],UNIT["degree",0.0174532925199433]],PROJECTION["Lambert_Conformal_Conic_2SP"],PARAMETER["standard_parallel_1",17.5],PARAMETER["standard_parallel_2",29.5],PARAMETER["latitude_of_origin",12],PARAMETER["central_meridian",-102],PARAMETER["false_easting",2500000],PARAMETER["false_northing",0]]'

In [94]:
gdf = gpd.GeoDataFrame.from_features([feature for feature in fc], 
                                      crs = var[1].geobox.crs.crs_str)


In [95]:
gdf.head()

,geometry,id
0,"POLYGON ((2977960.000 926120.000, 2977980.000 ...",537885
1,"POLYGON ((2978250.000 926120.000, 2978420.000 ...",537675
2,"POLYGON ((2978440.000 926120.000, 2978510.000 ...",537799
3,"POLYGON ((2978610.000 926120.000, 2978650.000 ...",537676
4,"POLYGON ((2979320.000 926120.000, 2979340.000 ...",537886


In [96]:
len(gdf.index)

538023

In [97]:
gdf = gdf.rename(columns={'id':'features_id'})

In [98]:
gdf.head()

,geometry,features_id
0,"POLYGON ((2977960.000 926120.000, 2977980.000 ...",537885
1,"POLYGON ((2978250.000 926120.000, 2978420.000 ...",537675
2,"POLYGON ((2978440.000 926120.000, 2978510.000 ...",537799
3,"POLYGON ((2978610.000 926120.000, 2978650.000 ...",537676
4,"POLYGON ((2979320.000 926120.000, 2979340.000 ...",537886


In [150]:
gdf.crs

'PROJCS["unnamed",GEOGCS["WGS 84",DATUM["unknown",SPHEROID["WGS84",6378137,6556752.3141]],PRIMEM["Greenwich",0],UNIT["degree",0.0174532925199433]],PROJECTION["Lambert_Conformal_Conic_2SP"],PARAMETER["standard_parallel_1",17.5],PARAMETER["standard_parallel_2",29.5],PARAMETER["latitude_of_origin",12],PARAMETER["central_meridian",-102],PARAMETER["false_easting",2500000],PARAMETER["false_northing",0]]'

**Intersección con área de interés**

In [99]:
aoi='/LUSTRE/MADMEX/tasks/2020/1_clusterization_for_agriculture_labeling/shp_puntos_martinez_simplify_lcc.shp'

In [100]:
gdf_aoi = gpd.read_file(aoi)

In [101]:
gdf_aoi.head()

,CVE_MUN,Municip,DstrtDR,Ecoregn,ZonadVd,Name,Obsrvcn,geometry
0,None,None,None,None,None,None,None,"MULTIPOLYGON (((2986835.636 925733.198, 298454..."


In [102]:
gdf_aoi.crs

{'proj': 'lcc',
 'lat_1': 17.5,
 'lat_2': 29.5,
 'lat_0': 12,
 'lon_0': -102,
 'x_0': 2500000,
 'y_0': 0,
 'datum': 'WGS84',
 'units': 'm',
 'no_defs': True}

https://gis.stackexchange.com/questions/266730/filter-by-bounding-box-in-geopandas

http://geopandas.org/set_operations.html

In [103]:
bbox = gdf.total_bounds

In [104]:
p1 = Point(bbox[0], bbox[3])
p2 = Point(bbox[2], bbox[3])
p3 = Point(bbox[2], bbox[1])
p4 = Point(bbox[0], bbox[1])
np1 = (p1.coords.xy[0][0], p1.coords.xy[1][0])
np2 = (p2.coords.xy[0][0], p2.coords.xy[1][0])
np3 = (p3.coords.xy[0][0], p3.coords.xy[1][0])
np4 = (p4.coords.xy[0][0], p4.coords.xy[1][0])

In [105]:
bb_polygon = Polygon([np1, np2, np3, np4])

In [106]:
gdf_bb = gpd.GeoDataFrame(gpd.GeoSeries(bb_polygon), columns=['geometry'])

In [107]:
gdf_bb

,geometry
0,"POLYGON ((2977960.000 926120.000, 3027980.000 ..."


In [108]:
gdf_aoi_intersect = gpd.overlay(gdf_bb,gdf_aoi,how='intersection')

In [109]:
gdf_aoi_intersect

,CVE_MUN,Municip,DstrtDR,Ecoregn,ZonadVd,Name,Obsrvcn,geometry
0,None,None,None,None,None,None,None,"MULTIPOLYGON (((3008631.202 926120.000, 302074..."


In [34]:
#gdf_aoi_intersect.plot() #descartes package needed for geopandas plot

In [110]:
start_time=time.time()
gdf_intersect = gpd.overlay(gdf,gdf_aoi_intersect,how='intersection')
end_time=time.time()

In [111]:
secs = end_time-start_time

In [112]:
secs

452.7877757549286

In [113]:
gdf_intersect.head()

,features_id,CVE_MUN,Municip,DstrtDR,Ecoregn,ZonadVd,Name,Obsrvcn,geometry
0,537885,None,None,None,None,None,None,None,"POLYGON ((2977960.000 926120.000, 2977980.000 ..."
1,537675,None,None,None,None,None,None,None,"POLYGON ((2978250.000 926120.000, 2978420.000 ..."
2,537799,None,None,None,None,None,None,None,"POLYGON ((2978440.000 926120.000, 2978510.000 ..."
3,537676,None,None,None,None,None,None,None,"POLYGON ((2978610.000 926120.000, 2978650.000 ..."
4,537886,None,None,None,None,None,None,None,"POLYGON ((2979320.000 926120.000, 2979340.000 ..."


In [114]:
len(gdf_intersect.index)

208008

In [115]:
gdf_intersect_reduced = gpd.GeoDataFrame()

In [116]:
gdf_intersect_reduced['features_id'] = gdf_intersect['features_id']
gdf_intersect_reduced['geometry'] = gdf_intersect['geometry']

In [117]:
gdf_intersect_reduced.head()

,features_id,geometry
0,537885,"POLYGON ((2977960.000 926120.000, 2977980.000 ..."
1,537675,"POLYGON ((2978250.000 926120.000, 2978420.000 ..."
2,537799,"POLYGON ((2978440.000 926120.000, 2978510.000 ..."
3,537676,"POLYGON ((2978610.000 926120.000, 2978650.000 ..."
4,537886,"POLYGON ((2979320.000 926120.000, 2979340.000 ..."


In [118]:
len(gdf_intersect_reduced.index)

208008

In [42]:
#start_time=time.time()
#gdf_intersect2 = gpd.overlay(gdf,gdf_aoi,how='intersection')
#end_time=time.time()

In [43]:
#secs = end_time-start_time

In [44]:
#secs

553.786865234375

In [148]:
gdf_intersect_reduced.crs

**Join df with fc in geopandas dataframe form**

In [119]:
gdf_df_join = gdf_intersect_reduced.merge(df2, on='features_id', how='left')

In [120]:
gdf_df_join.head()

,features_id,geometry,ndvi_mean,ndvi_min,ndvi_max
0,537885,"POLYGON ((2977960.000 926120.000, 2977980.000 ...",7511.008475,5017.644266,8728.280139
1,537675,"POLYGON ((2978250.000 926120.000, 2978420.000 ...",6973.082111,4899.799599,8439.416367
2,537799,"POLYGON ((2978440.000 926120.000, 2978510.000 ...",6930.858565,2623.810925,9064.791655
3,537676,"POLYGON ((2978610.000 926120.000, 2978650.000 ...",6209.316944,4308.028741,7732.793522
4,537886,"POLYGON ((2979320.000 926120.000, 2979340.000 ...",6841.346950,5490.895325,8038.980963


In [121]:
len(gdf_df_join.index)

208008

In [149]:
gdf_df_join.crs

# 5) Unsupervised clustering with Kmeans

In [122]:
nclusters=4

In [123]:
kmeans = KMeans(n_clusters=nclusters, n_jobs=10)

In [124]:
kmeans.fit(gdf_df_join[features].values)

KMeans(algorithm='auto', copy_x=True, init='k-means++', max_iter=300,
       n_clusters=4, n_init=10, n_jobs=10, precompute_distances='auto',
       random_state=None, tol=0.0001, verbose=0)

**Write model fit**

In [125]:
model_filename = path_result_dc_tile + date_today + '_model_' +  dc_tile + '_nclusters_%d' % nclusters + '.pkl'

In [126]:
model_filename

'/LUSTRE/MADMEX/tasks/2020/1_clusterization_for_agriculture_labeling/features_computed/ndvi_mean-ndvi_min-ndvi_max/02-01-2020/40_-31/02-01-2020_model_40_-31_nclusters_4.pkl'

In [127]:
pickle.dump(kmeans, open(model_filename, "wb"))

**Predict with model**

In [128]:
model_filename = path_result_dc_tile + date_today + '_model_' +  dc_tile + '_nclusters_%d' % nclusters + '.pkl'

In [129]:
model_filename

'/LUSTRE/MADMEX/tasks/2020/1_clusterization_for_agriculture_labeling/features_computed/ndvi_mean-ndvi_min-ndvi_max/02-01-2020/40_-31/02-01-2020_model_40_-31_nclusters_4.pkl'

In [130]:
kmeans = pickle.load(open(model_filename, "rb"))

**Predict**

In [131]:
preds = kmeans.predict(gdf_df_join[features].values)

In [132]:
preds

array([2, 2, 0, ..., 0, 0, 0], dtype=int32)

In [133]:
gdf_df_clusters = gdf_df_join.drop(features,axis=1)

In [134]:
gdf_df_clusters.head()

,features_id,geometry
0,537885,"POLYGON ((2977960.000 926120.000, 2977980.000 ..."
1,537675,"POLYGON ((2978250.000 926120.000, 2978420.000 ..."
2,537799,"POLYGON ((2978440.000 926120.000, 2978510.000 ..."
3,537676,"POLYGON ((2978610.000 926120.000, 2978650.000 ..."
4,537886,"POLYGON ((2979320.000 926120.000, 2979340.000 ..."


In [135]:
gdf_df_clusters['preds'] = preds

In [136]:
gdf_df_clusters.head()

,features_id,geometry,preds
0,537885,"POLYGON ((2977960.000 926120.000, 2977980.000 ...",2
1,537675,"POLYGON ((2978250.000 926120.000, 2978420.000 ...",2
2,537799,"POLYGON ((2978440.000 926120.000, 2978510.000 ...",0
3,537676,"POLYGON ((2978610.000 926120.000, 2978650.000 ...",2
4,537886,"POLYGON ((2979320.000 926120.000, 2979340.000 ...",2


In [137]:
len(gdf_df_clusters.index)

208008

# 6) Write results to file

**Geopackage**

In [138]:
path_results_geopackage = '/home/madmex_user/agriculture_geopackages/features_computed/' + features_string + '/'

In [139]:
path_results_geopackage

'/home/madmex_user/agriculture_geopackages/features_computed/ndvi_mean-ndvi_min-ndvi_max/'

In [140]:
path_result_geopackage = os.path.join(path_results_geopackage, date_today 
                                       + '/' + dc_tile + '/')

In [141]:
path_result_geopackage

'/home/madmex_user/agriculture_geopackages/features_computed/ndvi_mean-ndvi_min-ndvi_max/02-01-2020/40_-31/'

In [142]:
if not os.path.exists(path_result_geopackage):
    os.makedirs(path_result_geopackage)

In [168]:
clusters_filename = path_result_geopackage + date_today + '_clusters_' +  dc_tile + '_nclusters_%d' % nclusters + '.gpkg'

In [169]:
clusters_filename

'/home/madmex_user/agriculture_geopackages/features_computed/ndvi_mean-ndvi_min-ndvi_max/02-01-2020/40_-31/02-01-2020_clusters_40_-31_nclusters_4.gpkg'

In [170]:
layer_name = date_today + '_clusters_' +  dc_tile + '_nclusters_%d' % nclusters

In [171]:
layer_name

'02-01-2020_clusters_40_-31_nclusters_4'

In [147]:
gdf_df_clusters.crs

In [151]:
gdf_df_clusters.crs=gdf_aoi.crs

In [167]:
gdf_df_clusters.crs

{'proj': 'lcc',
 'lat_1': 17.5,
 'lat_2': 29.5,
 'lat_0': 12,
 'lon_0': -102,
 'x_0': 2500000,
 'y_0': 0,
 'datum': 'WGS84',
 'units': 'm',
 'no_defs': True,
 'wktext': True}

In [176]:
#try:
#    string_wkt = gdf_df_clusters.crs
#    srs = osr.SpatialReference()
#    srs.ImportFromWkt(string_wkt)
#    srs.MorphToESRI()
#    p = Proj(srs.ExportToWkt())
#except:
#    p = Proj(gdf_df_clusters.crs)

In [177]:
#if not p.crs.is_geographic:
#    gdf_df_clusters.crs = from_epsg(4326)
#    gdf_df_clusters.to_crs(gdf_df_clusters.crs)

In [212]:
#gdf_df_clusters.crs = from_epsg(4326)

In [172]:
start_time=time.time()

gdf_df_clusters.to_file(clusters_filename,
                        layer=layer_name,
                        driver='GPKG')
end_time=time.time()

In [173]:
secs = end_time-start_time
secs

98.28087639808655

In [174]:
shutil.move(clusters_filename,path_result_dc_tile)

'/LUSTRE/MADMEX/tasks/2020/1_clusterization_for_agriculture_labeling/features_computed/ndvi_mean-ndvi_min-ndvi_max/02-01-2020/40_-31/02-01-2020_clusters_40_-31_nclusters_4.gpkg'

**Shapefile**

In [156]:
clusters_filename = path_result_dc_tile + date_today + '_clusters_' +  dc_tile + '_nclusters_%d' % nclusters + '.shp'

In [157]:
clusters_filename

'/LUSTRE/MADMEX/tasks/2020/1_clusterization_for_agriculture_labeling/features_computed/ndvi_mean-ndvi_min-ndvi_max/02-01-2020/40_-31/02-01-2020_clusters_40_-31_nclusters_4.shp'

In [162]:
gdf_df_clusters.crs=gdf_aoi.crs

In [163]:
gdf_df_clusters.crs

{'proj': 'lcc',
 'lat_1': 17.5,
 'lat_2': 29.5,
 'lat_0': 12,
 'lon_0': -102,
 'x_0': 2500000,
 'y_0': 0,
 'datum': 'WGS84',
 'units': 'm',
 'no_defs': True}

In [191]:
#try:
#    string_wkt = gdf_df_clusters.crs
#    srs = osr.SpatialReference()
#    srs.ImportFromWkt(string_wkt)
#    srs.MorphToESRI()
#    p = Proj(srs.ExportToWkt())
#except:
#    p = Proj(gdf_df_clusters.crs)

In [192]:
#if not p.crs.is_geographic:
#    gdf_df_clusters.crs = from_epsg(4326)
#    gdf_df_clusters.to_crs(gdf_df_clusters.crs)

In [193]:
#gdf_df_clusters.crs

{'init': 'epsg:4326', 'no_defs': True}

In [164]:
#gdf_df_clusters.to_crs(gdf_df_clusters.crs)

In [165]:
start_time=time.time()
gdf_df_clusters.to_file(clusters_filename)
end_time=time.time()

In [166]:
secs = end_time-start_time
secs

178.3230493068695